In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import openpyxl
import statistics
import math
import os
from matplotlib.backends.backend_pdf import PdfPages


def diff_x_i(sig, x_i) :
    return [sig[i]-x_i for i in range(len(sig))]


def diff_a_iw_abs(a_i, a_iw) :
    return abs(a_i - a_iw)



    
    
    
#ファイルの読み込み                                                                                                                         
#file_name1 = sys.argv[1]                                                                                                                   
#file_name2 = sys.argv[2]                                                                                                                   
#file1 = pd.ExcelFile(file_name1+'.xlsx')                                                                                                   
#file2 = pd.ExcelFile(file_name2+'.xlsx')                                                                                                   

def write_data(wb_sheet, sig, column_point, start_row, value_title) :
    wb_sheet.cell(column=column_point, row=1, value=value_title)
    for i, sig_data in enumerate(sig) :
        #print(sig_data)
        wb_sheet.cell(column=column_point, row=i+start_row, value=sig_data)

# ディレクトリ作成
def my_makedirs(path):
    if not os.path.isdir(path):
        os.makedirs(path)
        
def list_mean_num(sig_mean, x) :
#    for i in range(x) :
#        sig_mean[i] = sig_mean[i]/x
    return [i/x for i in sig_mean]
    
    

In [218]:
def inspect(file1, k, h_start, peak, sig_mean_mean, sig_sd_mean, spike_plot=True, type="nomal"):
    #ファイル1のデータカウント                                                                                                              
    pattern_dict1 = {}
    sumpsth = 0
    mean_frag = 0
    for i, name in enumerate(sheet_names1):
#        print(wb[name]['volt'])
        sheet_df1[i] = file1.parse(name)
        #print(sheet_df1[i])
        if (sig_mean_mean[0] == 0.0) :
            mean_frag = 1
        
        if(spike_plot == True) :
            try :
                sig_ori = (sheet_df1[i]["volt"]).values
                ripple_num = i
            except KeyError :
                print("not max data number")
                ripple_num = i
                break
            for j in range(10) :
                S_1 = [(max(diff_x_i(sig_ori[i-k:i], sig_ori[i])) + max(diff_x_i(sig_ori[i+1:i+k+1], sig_ori[i])))/2 for i in range(k, len(sig_ori)-k)]
                if(mean_frag == 1) :
                    sig_mean_mean[j] += sum(S_1)/len(S_1) # m'
                    sig_sd_mean[j] += statistics.pstdev(S_1) # s'
            #print(sig_mean_mean[0])
            #print(statistics.pstdev(S_1))
            #print(sig_sd_mean[0])
    #print(ripple_num)
    if(mean_frag == 1) :
        sig_mean_mean = list_mean_num(sig_mean_mean, ripple_num+1)
        sig_sd_mean = list_mean_num(sig_sd_mean, ripple_num+1)
    #print(sig_sd_mean)
    #return sig_mean_mean, sig_sd_mean
    #print(sig_sd_mean)  
    #print()
    
    for i, name in enumerate(sheet_names1) :
        try :
#            for row in wb[name].columns :
#                sig_ori = []
#                for cell in row : 
                    #print(cell.value)
#                    sig_ori.append(cell.value)
            sig_ori = (sheet_df1[i]["volt"]).values
        except KeyError :
            print("not max data number")
            break
        
        sig_smooth = [(sig_ori[i]+sig_ori[i+1]+sig_ori[i+2])/3  for i in range(len(sig_ori)-2)]
#        write_data(file1[name], sig_smooth, 2, 3, "smooth")
            
        sig_fire = [ 1 if(sig_smooth[i-3] > sig_smooth[i-2]) & (sig_smooth[i-2] > sig_smooth[i-1]) & (sig_smooth[i-1] > sig_smooth[i]) \
                   & (sig_smooth[i] < sig_smooth[i+1]) & (sig_smooth[i+1] < sig_smooth[i+2]) & (sig_smooth[i+2] < sig_smooth[i+3]) & (sig_ori[i+1] <= peak) \
                    else 0 for i in range(3, len(sig_smooth)-3)]
        
        sig_fire_a = [ 1 if(sig_smooth[i] - sig_smooth[i-1] <= 0) & (sig_smooth[i+1] - sig_smooth[i] >= 0) & (sig_ori[i+1] <= peak) \
                     else 0 for i in range(3, len(sig_smooth) -3)]
        
        
        
        plot_len = len(sig_ori) - (2*k)
        time_plot = [i/25 for i in range(plot_len)]
        peak_line = [peak for i in range(plot_len)]
        plt.figure(figsize=(10, 4))
        
        if(spike_plot == True) :
            plt.subplot(3, 1, 1)
            plt.subplots_adjust(wspace=0.0, hspace=0)
        #plt.figure()
        start_point = int((len(sig_ori)-plot_len)/2)
        if(type=="short") :
            plt.xlim(20, 40)
        else :
            plt.xlim(0, plot_len/25)
        plt.xticks(color="None")
        if(start_point == 0) :
            plt.plot(time_plot, sig_ori, linewidth = 0.5)
        else :
            plt.plot(time_plot, sig_ori[start_point:-start_point], linewidth = 0.5)
            plt.plot(time_plot, peak_line, color="orange", linewidth = 0.5)            
            
        plt.subplot(21, 1, 8)
        plt.ylim(0.95, 1.05)
        plt.xticks(color="None")
        plt.yticks(color="None")
        #plt.scatter([i/25 for i in range(len(sig1))],sig1,marker='s', s=10)
        start_point = int((len(sig_fire_a)-plot_len)/2)
        plt.xlim(0, plot_len/25)
        if(type=="short") :
            plt.xlim(20, 40)
        else :
            plt.xlim(0, plot_len/25)
        if(start_point == 0) :
            plt.scatter([i/25 for i in range(len(sig_fire_a))], sig_fire_a, color="orange", marker='s', s=3)
        else :
            plt.scatter([i/25 for i in range(len(sig_fire_a[start_point:-start_point]))], sig_fire_a[start_point:-start_point], color="orange", marker='s', s=3)
   
        
        if(spike_plot == True) :       
            for j in range(10) :
                h = h_start + (j/10)
                #h = 95+j
                S_1 = [(max(diff_x_i(sig_ori[i-k:i], sig_ori[i])) + max(diff_x_i(sig_ori[i+1:i+k+1], sig_ori[i])))/2 for i in range(k, len(sig_ori)-k)]
                #sig_mean = sum(sig_ori)/len(sig_ori) # m'
                #sig_sd = statistics.pstdev(sig_ori) # s'
                S_1_fire = [1 if(S_1[i]-sig_mean_mean[j] > h*sig_sd_mean[j]) & (sig_ori[i+k] < sig_ori[i+k-1]) & (sig_ori[i+k] < sig_ori[i+k+1]) & (sig_ori[i+k] <= peak) else 0 for i in range(len(S_1))]

                plt.subplot(21, 1, j+9)
                plt.ylim(0.95, 1.05)
                if(type=="short") :
                    plt.xlim(20, 40)
                else :
                    plt.xlim(0, plot_len/25)
                plt.yticks(color="None")
                if(j != 9) :
                    plt.xticks(color="None")
                #plt.scatter([i/25 for i in range(len(sig1))],sig1,marker='s', s=10)
                start_point = int((len(S_1_fire)-plot_len)/2)
                if(start_point == 0) :
                    plt.scatter([i/25 for i in range(len(S_1_fire))], S_1_fire,marker='s', s=3)
                else :
                    plt.scatter([i/25 for i in range(len(S_1_fire[start_point:-start_point]))], S_1_fire[start_point:-start_point],marker='s', s=3)          
        pdf.savefig()
        plt.close()  
    #plt.scatter([i/25 for i in range(len(S_1_fire))], S_1_fire,marker='s', s=10)
    #print(S_1_fire)
    return sig_mean_mean, sig_sd_mean

    


In [55]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/male-25000/B38-HR/B38-male-R2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-mean-data-short/"
save_path = "/home/nodoka/win2/data4.0/send-mean-data/"
ep_individual = "male-25000/B38-HR/"
timing_data = "B38-male-R3"

In [66]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/female-25000/C14-HL/C14-female-L2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-mean-data-short/"
save_path = "/home/nodoka/win2/data4.0/send-mean-data/"
ep_individual = "female-25000/C14-HL/"
timing_data = "C14-female-L3"

In [55]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/female-25000/B31-HR/B31-female-R2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-mean-data-short/"
save_path = "/home/nodoka/win2/data4.0/send-mean-data/"
ep_individual = "female-25000/B31-HR/"
timing_data = "B31-female-R3"

In [234]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/restraint-25000/C15-HR/C15-restraint-R2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-line-data-short/"
#save_path = "/home/nodoka/win2/data4.0/send-line-data/"
ep_individual = "restraint-25000/C15-HR/"
timing_data_file = "C15-restraint-R"
line = -0.1

In [240]:
#file1 = pd.ExcelFile(sys.argv[1])
#file1 = pd.ExcelFile("/home/nodoka/win/ubuntu/01-data/restraint-25000/B39-HR/B39-restraint-R2.xlsx")
#file1 = pd.ExcelFile("C14-female-L2.xlsx")
file1 = pd.ExcelFile("/home/nodoka/win2/data4.0/all-newtype-bp-ripple/object-25000/B32-HL/B32-object-L2.xlsx")
read_path = "/home/nodoka/win2/data4.0/all-newtype-bp-ripple/"
save_path = "/home/nodoka/win2/data4.0/send-line-data-short/"
#save_path = "/home/nodoka/win2/data4.0/send-line-data/"
ep_individual = "object-25000/B32-HL/"
timing_data_file = "B32-object-L"
line = -0.45

In [246]:
wb = openpyxl.load_workbook("peak-line.xlsx")
ws = wb.worksheets[0]

for row in ws.iter_rows(min_row=1):
    values=[]
    for col in row:
        values.append(col.value)
    if(values[0] == timing_data_file) :
        line = values[1]
    #print(values)

In [225]:
for h in range(1,3):
    for k in range(4, 7) :
        for timing in range(2, 4) :
            timing_data = timing_data_file + str(timing)
            file1 = pd.ExcelFile(read_path + ep_individual + timing_data + ".xlsx")
            sheet_df1 = file1.parse(file1.sheet_names, header=None)
            sheet_names1 = file1.sheet_names
            my_makedirs(save_path+ep_individual+timing_data)

            pdf = PdfPages(save_path + ep_individual + timing_data + "/parameta_" + str(h) + "h_" + str(k) + "k.pdf")
            if(timing == 2) :            
                sig_mean_mean = [0.0 for j in range(10)]
                sig_sd_mean = [0.0 for j in range(10)]
                sig_mean_mean, sig_sd_mean = inspect(file1, k, h, line, sig_mean_mean, sig_sd_mean, True, "short")
                #sig_mean_mean, sig_sd_mean = inspect(file1, k, h, line, sig_mean_mean, sig_sd_mean, True)
            else :
                inspect(file1, k, h, line, sig_mean_mean, sig_sd_mean, True, "short")
                #inspect(file1, k, h, line, sig_mean_mean, sig_sd_mean, True)
            pdf.close()
            print(timing)

2
3
2
3
2
3
2
3
2
3
2
3
